In [77]:
import numpy as np
import matplotlib.pyplot as plt
import xtrack as xt
import xpart as xp
import xcoll as xc
import xobjects as xo

In [78]:
def install_tidp(line, block_mvt=29e-3):
    tidp_ap_tot = 147e-3
    line.discard_tracker()
    tidp = xc.EverestCollimator(length=4.3, material=xc.materials.Carbon, jaw_L= tidp_ap_tot/2 + block_mvt, jaw_R = -tidp_ap_tot/2 + block_mvt)
    line.collimators.install(names=['tidp.11434'], elements=[tidp])
    return tidp

In [84]:
context = xo.ContextCpu()
line = xt.Line.from_json('../../injection_lines/sps_with_aperture_inj_q20_beam_sagitta5.json')

line.vars['qph_setvalue'] = 0.5
line.vars['qpv_setvalue'] = 0.5
line.vars['qh_setvalue'] = line.vars['qx0']._value + 0.05
line.vars['qv_setvalue'] = line.vars['qy0']._value + 0.05


cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)

for name in cavity_names:
    line[name].frequency = 200e6
    line[name].lag = 180
line['acl.31735'].voltage = 0 #setting 800 cav to 0V
line['actcse.31632'].voltage = 3.0e6

tt = line.get_table()
tw = line.twiss()

Loading line from dict:   0%|          | 0/36404 [00:00<?, ?it/s]

Done loading line from dict.           


In [85]:
context = xo.ContextCpu()

aperture_mvt = 0
i = 329
num_particles  = 1000
plane = 'DPneg'
sweep = 6000
sweep = -abs(sweep) if plane == 'DPpos' else abs(sweep)
num_turns = 6000

In [86]:
# Changing apertures of loss locations
line['bpcn.12508.a_aper'].shift_x = aperture_mvt
line['bpcn.12508.b_aper'].shift_x = aperture_mvt


line['bpcn.61108.a_aper'].shift_x = aperture_mvt
line['bpcn.61108.b_aper'].shift_x = aperture_mvt

In [87]:
tidp = install_tidp(line)
tw = line.twiss()
part = xp.generate_matched_gaussian_bunch(nemitt_x=2e-6,
                                          nemitt_y=2e-6,
                                          sigma_z=0.224, num_particles=num_particles, line=line)

Slicing line:   0%|          | 0/33012 [00:00<?, ?it/s]

*** Maximum RMS bunch length 0.23610110213554214m.
... distance to target bunch length: -2.2226e-01
... distance to target bunch length: 5.5784e-03
... distance to target bunch length: 5.2361e-03
... distance to target bunch length: -6.3267e-03
... distance to target bunch length: 1.9948e-03
... distance to target bunch length: -3.8258e-04
... distance to target bunch length: 5.3833e-05
... distance to target bunch length: 1.2594e-06
... distance to target bunch length: -1.0995e-10
... distance to target bunch length: 1.2805e-07
--> Bunch length: 0.22399999989004873
--> Emittance: 0.28106907515669655


In [88]:
rf_sweep = xc.RFSweep(line)
rf_sweep.info(sweep=sweep, num_turns=num_turns)

line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu(omp_num_threads='auto'))
line.scattering.enable()
rf_sweep.track(sweep=sweep, particles=part, num_turns=num_turns, time=True, with_progress=True, ele_start = 0)
line.scattering.disable()

The current frequency is 200000000.0Hz, adding 1.0Hz per turn until 200006000.0 (for 6000 turns).
This sweep will move the center of the bucket with Δδ = -0.016896947679657257 (-2.81975385240468 buckets).
Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored
ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored


Done compiling ContextCpu kernels.


Tracking:   0%|          | 0/6000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
len(line.element_names)

471.6

In [ ]:
line.get_length()/100

np.float64(6911.50379999983)

In [16]:
6900/100

69.0

In [19]:
tt = line.get_table()

In [20]:
tt.name

array(['sps$start', 'begi.10010', 'qf.10010', ..., 'end.10010', 'sps$end',
       '_end_point'], shape=(33013,), dtype=object)

In [29]:
len(line.element_names)

33012

In [ ]:
for i in range(330):
    if i == 329:

330.12

In [33]:
329*100 + 1

32901

In [35]:
33012/100

330.12

In [36]:
329 *100+1 < 33012

True

In [75]:
np.sqrt(2500)

np.float64(50.0)

In [76]:
400 *31

12400